In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from Model.demanda import DemandaAtual, DemandaAcumulada, CapacidadeOperacional
from Model.analista import Analista
import Control.manager_data as Data_Man
from Control.calculadora import Calculadora

In [ ]:
def modelo_01(tma, demanda_atual:DemandaAtual, demanda_acumulada:DemandaAcumulada, capacidade_operacional:CapacidadeOperacional):
    PICO_MANHA = 10
    PICO_NOITE = 21
    
    analistas = []

    while demanda_acumulada.get_demanda()[PICO_MANHA] > 0:
        novo_analista = Analista(tma, "07:00", "12:00", "15:20")
        analistas.append(novo_analista)
        
        demanda_atualizada = [a - b for a, b in zip(demanda_atual.get_demanda(), novo_analista.get_capacidade_producao())]
        
        demanda_atual.set_demanda(demanda_atualizada)
        demanda_acumulada.recalcular_acumulo(demanda_atualizada)
        
    while demanda_acumulada.get_demanda()[PICO_NOITE] > 0:
        novo_analista = Analista(tma, "13:40", "17:25", "22:00")
        analistas.append(novo_analista)
        
        demanda_atualizada = [a - b for a, b in zip(demanda_atual.get_demanda(), novo_analista.get_capacidade_producao())]
        
        demanda_atual.set_demanda(demanda_atualizada)
        demanda_acumulada.recalcular_acumulo(demanda_atualizada)
    
    capacidade_producao_atualizada = [0] * 24
    for analista in analistas:
        capacidade = analista.get_capacidade_producao()
        capacidade_producao_atualizada = [a + b for a, b in zip(capacidade_producao_atualizada, capacidade)]
        
    capacidade_operacional.set_capacidade_producao(capacidade_producao_atualizada)

    return analistas

In [ ]:
df_csv_der = Data_Man.get_dataframe(r'../Data/dericavao_24h.csv')                
calculadora = Calculadora()
                
(demanda_atual, 
    demanda_acumulada, 
    capacidade_operacional) = calculadora.create_instancias()

demanda_inicial = df_csv_der['quantidade'].tolist()

demanda_atual.set_demanda(demanda_inicial)

acumulo = calculadora.calcular_acumulo(demanda_inicial)
acumulo[22] = 0
acumulo[23] = 0

demanda_acumulada.set_demanda(acumulo)

analistas = modelo_01(156, demanda_atual, demanda_acumulada, capacidade_operacional)